In [1]:
import pandas as pd
import xlrd
from datetime import datetime, timedelta
from scipy.io import wavfile
import os
import urllib.request
from tqdm.autonotebook import tqdm

/home/AD/tsainbur/anaconda/lib/python3.7/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


### read bird excel file

In [2]:
song_db = pd.read_excel('BIRD_DB.xls')
mainData_book = xlrd.open_workbook("BIRD_DB.xls", formatting_info=True)
mainData_sheet = mainData_book.sheet_by_index(0)
song_urls = ['' if mainData_sheet.hyperlink_map.get((i, 11)) == None else mainData_sheet.hyperlink_map.get((i, 11)).url_or_path for i in range(mainData_sheet.nrows)]
song_db['Audio_file'] = song_urls[1:]
song_db = song_db[1:]

In [3]:
song_db[:3]

TrackName     type_of_device configuration  \
1  tCOMM09-1  single microphone       MOC2012   
2  tCOMM09-2  single microphone       MOC2012   
3  tCOMM09-3  single microphone       MOC2012   

                                          microphone        recorder  \
1  Sennheiser omnidirectional with Telinga parabo...  Marantz PMD650   
2  Sennheiser omnidirectional with Telinga parabo...  Marantz PMD650   
3  Sennheiser omnidirectional with Telinga parabo...  Marantz PMD650   

   sample_rate Recordist_first_name Recordist_last_name recording_date  \
1         44.0               Martin                Cody     2009-03-21   
2         44.0               Martin                Cody     2009-03-21   
3         44.0               Martin                Cody     2009-03-21   

  recording_time                        ...                          Region  \
1       08:25:00                        ...                          Amador   
2       08:27:00                        ...                          Amador   
3       08:28:00                        ...                          Amador   

  country state_or_province vegetation_type        Analysis Analysis_date  \
1     USA        California  Open Chapparal  CATH processed    2013-02-23   
2     USA        California  Open Chapparal  CATH processed    2013-02-23   
3     USA        California  Open Chapparal  CATH processed    2013-02-23   

    Analysis_method number_of_phrases  \
1  Praat annotation             383.0   
2  Praat annotation             152.0   
3  Praat annotation             140.0   

                               Textgrid_file  \
1  Files_TextGrids/2009/March/CATH1.TextGrid   
2  Files_TextGrids/2009/March/CATH2.TextGrid   
3  Files_TextGrids/2009/March/CATH3.TextGrid   

                                        Keycode_file  
1  Files_Keys/CATH_2009/TRIPLELETTER_N_CA_phrases...  
2  Files_Keys/CATH_2009/TRIPLELETTER_N_CA_phrases...  
3  Files_Keys/CATH_2009/TRIPLELETTER_N_CA_phrases...  

[3 rows x 44 columns]

In [4]:
print(len(song_db),mainData_sheet.nrows)

645 647


### Download waveforms and TextGrids

In [5]:
bird_db_loc = '../../../data/bird-db'
if not os.path.exists(bird_db_loc):
    os.makedirs(bird_db_loc)

In [6]:
from urllib.error import HTTPError
from sklearn.externals.joblib import Parallel, delayed

In [7]:
parallel = True
verbosity=0
n_jobs=20
save=True

In [8]:
def downloadBirdDB(row):
    wav = row['Audio_file']
    text_grid = row['Textgrid_file']
    track_name = row['TrackName']
    subject_id = row['SubjectName']
    species = row['Species_short_name']
    recording_time = row['recording_date'] + timedelta(hours = row['recording_time'].hour,
                                                          minutes = row['recording_time'].minute,
                                                          seconds = row['recording_time'].second)
    # PREP SAVE LOCATION
    wav_location = '/'.join([bird_db_loc,species,subject_id,'wavs', recording_time.strftime("%Y-%m-%d_%H-%M-%S-%f")+'.wav'])
    grid_location = '/'.join([bird_db_loc,species,subject_id,'TextGrids', recording_time.strftime("%Y-%m-%d_%H-%M-%S-%f")+'.TextGrid'])
    if not os.path.exists('/'.join([bird_db_loc,species,subject_id,'wavs'])): 
        os.makedirs('/'.join([bird_db_loc,species,subject_id,'wavs'])) 
    if not os.path.exists('/'.join([bird_db_loc,species,subject_id,'TextGrids'])):                              
        os.makedirs('/'.join([bird_db_loc,species,subject_id,'TextGrids'])) 
    
    # save wav
    if not os.path.exists(wav_location): 
        try:
            urllib.request.urlretrieve(wav, wav_location)
        except HTTPError:
            print('Could not retreive ' + wav)
    # save textgrid
    if not os.path.exists(grid_location): 
        try:
            urllib.request.urlretrieve('http://taylor0.biology.ucla.edu/birdDBQuery/Files/'+text_grid, grid_location)
        except HTTPError:
            print('Could not retreive ' + 'http://taylor0.biology.ucla.edu/birdDBQuery/Files/'+text_grid)

In [ ]:
if parallel:
    with Parallel(n_jobs=n_jobs, verbose=verbosity) as parallel:
        parallel(delayed(downloadBirdDB)(row) 
            for idx, row in tqdm(song_db.iterrows(), total=len(song_db)))
else:
    for idx, row in tqdm(song_db.iterrows(), total=len(song_db)):
        downloadBirdDB(row) 


Could not retreive http://taylor0.biology.ucla.edu/birdDBQuery/Files/Tracks/2014/May/1454.WAV
Could not retreive http://taylor0.biology.ucla.edu/birdDBQuery/Files/Files_TextGrids/2014/May/1497.TextGrid
Could not retreive http://taylor0.biology.ucla.edu/birdDBQuery/Files/Files_TextGrids/2014/May/1454.TextGrid
Could not retreive http://taylor0.biology.ucla.edu/birdDBQuery/Files/Files_TextGrids/2013/May/1023.TextGrid
Could not retreive http://taylor0.biology.ucla.edu/birdDBQuery/Files/Files_TextGrids/2013/Jun/1222.TextGrid
Could not retreive http://taylor0.biology.ucla.edu/birdDBQuery/Files/Files_TextGrids/2013/May/1022.TextGrid
Could not retreive http://taylor0.biology.ucla.edu/birdDBQuery/Files/Files_TextGrids/2013/Jun/1257.TextGrid
Could not retreive http://taylor0.biology.ucla.edu/birdDBQuery/Files/Files_TextGrids/2013/May/1021.TextGrid

Could not retreive http://taylor0.biology.ucla.edu/birdDBQuery/Files/Files_TextGrids/2014/Jun/1604.TextGrid
Could not retreive http://taylor0.biology